In [2]:
import sys
import os
from glob import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle as pkl
import importlib
from ppca import PPCA

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import RFECV

import utils
from utils.testers import ss, assess
from utils.reporters import basic_scores, bs_rankings, bs_curves
from utils.helpers import mask50, myLabelEncoder, tlabels, slabels, plabels
importlib.reload(utils.reporters)
importlib.reload(utils.testers)
importlib.reload(utils.helpers)

<module 'utils.helpers' from '/home/dvolo/uni/00-landoni/bigmac/utils/helpers.py'>

In [3]:

    

## Dataset import
raw_data_path = "data/pkl/divided/"

tf = pd.read_pickle(raw_data_path+"transient_features.pkl")
tl = pd.read_pickle(raw_data_path+"transient_labeled.pkl")
sf = pd.read_pickle(raw_data_path+"stochastic_features.pkl")
sl = pd.read_pickle(raw_data_path+"stochastic_labeled.pkl")
pf = pd.read_pickle(raw_data_path+"periodic_features.pkl")
pl = pd.read_pickle(raw_data_path+"periodic_labeled.pkl")

tme = myLabelEncoder(tlabels)
sme = myLabelEncoder(slabels)
pme = myLabelEncoder(plabels)

si = SimpleImputer(strategy="constant",fill_value=-999)

Xt = si.fit_transform(tf.values)
Xs = si.fit_transform(sf.values)
Xp = si.fit_transform(pf.values)

yt = tme.fit_transform(tl["classALeRCE"].values)
ys = sme.fit_transform(sl["classALeRCE"].values)
yp = pme.fit_transform(pl["classALeRCE"].values)

XXp = np.load("data/npy/ppcas3/X/basicfull.npy")

In [4]:
# compute_save_rfecv(Xt, yt, "data/rfe/rfe-transient.pkl")
# compute_save_rfecv(Xs, ys, "data/rfe/rfe-stochastic.pkl")
compute_save_rfecv(Xp, yp, "data/pkl/rfe/rfe-periodic.pkl", step=1, verbose=0)

KeyboardInterrupt: 

In [ ]:
# compute_save_rfecv(Xt, yt, "data/rfe/rfe-transient.pkl")
# compute_save_rfecv(Xs, ys, "data/rfe/rfe-stochastic.pkl")
compute_save_rfecv(Xp, yp, "data/pkl/rfe/rfe-periodic.pkl", step=1, verbose=0)